In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install japanize_matplotlib bottleneck tslearn

In [24]:
#作った関数
import MasterResearchFunction as mr
# 基本ライブラリ
import os, re, csv, math, statistics
from datetime import datetime, timedelta
from decimal import Decimal

# 数値計算とデータ処理
import numpy as np
import pandas as pd
import bottleneck as bn

# 機械学習ライブラリ
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# ディープラーニングライブラリ
import tensorflow as tf
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation, Conv1D, MaxPooling1D, Flatten

# プロットと可視化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import japanize_matplotlib

# その他のライブラリ
from scipy import signal, stats
from scipy.signal import savgol_filter
from tslearn.metrics import dtw_path

In [25]:
motion_data = mr.process_apple_watch_csv("datasets/yuuma/20240604/yuuma_motion.csv")
#motion_data = mr.process_apple_watch_csv("/content/drive/MyDrive/datasets/kawano/20240517/MotionData_20240517_220252.csv")

In [ ]:
motion_data

In [ ]:
check = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_check")
circle1 = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_circle")
cross = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_cross")
tri = mr.process_all_apple_watch_csv_in_directory("datasets/kawano/train_gesture/kawano_tri")
# check = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_check")
# circle = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_circle")
# cross = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_cross")
# tri = mr.process_all_apple_watch_csv_in_directory("/content/drive/MyDrive/datasets/kawano/train_gesture/kawano_tri")

In [26]:
eye_data = mr.process_tobii_csv("datasets/yuuma/20240604/yuuma_eye.csv")

/Users/hinase/CodeChord/MasterResearch/MasterResearchFunction.py:51: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  eye_data = pd.read_csv(file_path, header=0)


In [ ]:
eye_data

In [27]:
# 指定された範囲のデータをフィルタリング
circle_start_time = '2024-06-04 16:00:31'
circle_end_time = '2024-06-04 16:00:34'
cross_start_time = '2024-06-04 16:10:56'
cross_end_time = '2024-06-04 16:10:58'
tri_start_time = '2024-06-04 16:31:37'
tri_end_time = '2024-06-04 16:31:40'
check_start_time = '2024-06-04 16:46:43'
check_end_time = '2024-06-04 16:46:45'
circle = motion_data[(motion_data['Timestamp'] >= circle_start_time) & (motion_data['Timestamp'] <= circle_end_time)].reset_index(drop=True)
cross = motion_data[(motion_data['Timestamp'] >= cross_start_time) & (motion_data['Timestamp'] <= cross_end_time)].reset_index(drop=True)
tri = motion_data[(motion_data['Timestamp'] >= tri_start_time) & (motion_data['Timestamp'] <= tri_end_time)].reset_index(drop=True)
check = motion_data[(motion_data['Timestamp'] >= check_start_time) & (motion_data['Timestamp'] <= check_end_time)].reset_index(drop=True)

In [ ]:
motion_data['Timestamp']

In [49]:
Th = 3

In [50]:
segx = mr.spring_ogawa(motion_data['AccelerationX'], cross['AccelerationX'], Th)
segy = mr.spring_ogawa(motion_data['AccelerationY'], cross['AccelerationY'], Th)
segz = mr.spring_ogawa(motion_data['AccelerationZ'], cross['AccelerationZ'], Th)

****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****
****


In [51]:
# Combine all segments
all_segments = []
for seg in [segx, segy, segz]:
    for (i, d_min, t_s, t_e) in seg:
        all_segments.append((t_s, t_e))

# Sort segments
all_segments.sort()

# Find overlapping segments
overlap_ranges = []
current_overlap = None
current_count = 0

for start, end in all_segments:
    if current_overlap is None:
        current_overlap = (start, end)
        current_count = 1
    else:
        current_start, current_end = current_overlap
        if start <= current_end:
            current_overlap = (current_start, max(current_end, end))
            current_count += 1
        else:
            if current_count >= 2:
                overlap_ranges.append(current_overlap)
            current_overlap = (start, end)
            current_count = 1

if current_count >= 2:
    overlap_ranges.append(current_overlap)

In [52]:
overlap_ranges

[(581, 639),
 (1100, 1138),
 (1330, 1379),
 (8048, 8156),
 (9132, 9194),
 (15218, 15316),
 (18356, 18450),
 (22163, 22244),
 (22497, 22616),
 (25013, 25109),
 (25531, 25692),
 (26795, 26900),
 (27027, 27108),
 (27344, 27490),
 (28509, 28585),
 (30564, 30662),
 (32851, 32913),
 (35121, 35195),
 (38695, 38803),
 (41028, 41127),
 (73461, 73519)]

In [44]:
circle35 = overlap_ranges

In [48]:
motion_data['Timestamp'][9909]

Timestamp('2024-06-04 16:00:33.992598016')

In [ ]:
cross3 = overlap_ranges

In [ ]:
tri35[36]

In [ ]:
check35

In [ ]:
motion_data['Timestamp'][66190]

In [ ]:
gesture_eye_data = []

In [ ]:
results = []
for start, end in circle35:
    a = [motion_data['Timestamp'][start], motion_data['Timestamp'][end]]
    results.append(a)

In [ ]:
len(results)

In [ ]:
for i in range(len(results)):
  gesture_eye_data.append(eye_data[(eye_data['Timestamp'] >= (results[i][0] - pd.Timedelta(seconds=5))) & (eye_data['Timestamp'] <= (results[i][1]) + pd.Timedelta(seconds=5))])

In [ ]:
len(gesture_eye_data)

In [ ]:
plt.figure(figsize=(72, 6))
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
plt.plot(gesture_eye_data[36]['Timestamp'], gesture_eye_data[36]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
# plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
plt.grid(True)  # グリッド表示
plt.legend()  # 凡例表示
plt.xticks(rotation=45)  # x軸のラベルを45度回転
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
# ハイライト
plt.axvspan(results[36][0], results[36][1], color='yellow', alpha=0.3)
plt.show()

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()

In [ ]:
for i in range(len(gesture_eye_data)):
    plt.figure(figsize=(72, 6))
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point X'].diff(), label='Gaze point X diff')  # 時系列データのプロット
    # plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Gaze point Y'].diff(), label='Gaze point Y diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter right'].diff(), label='Pupil diameter right diff')  # 時系列データのプロット
    plt.plot(gesture_eye_data[i]['Timestamp'], gesture_eye_data[i]['Pupil diameter left'].diff(), label='Pupil diameter left diff')  # 時系列データのプロット
    plt.grid(True)  # グリッド表示
    plt.legend()  # 凡例表示
    plt.xticks(rotation=45)  # x軸のラベルを45度回転
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%M:%S:%f'))  # Timestampフォーマット設定
    # ハイライト
    plt.axvspan(results[i][0], results[i][1], color='yellow', alpha=0.3)
    plt.show()